In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import spacy

In [602]:
import re

def prep_input(series):
    nlp = spacy.load("sv_core_news_sm")
    

    regexp4 = r'\b(?<![\\\.])\w+(?!\.\w+)\b'
    result_list = []
    token_list = []
    for row in nlp.pipe(series):
        res = BeautifulSoup(row.text, 'lxml').get_text()
        res = re.findall(regexp4, res)
        token_list.append([token.lemma_.lower() for token in row if not token.is_stop 
                                                       and token.is_alpha])
    
    #display(pd.Series(token_list))
    return pd.Series(token_list)

def preprocess_input(input):
    nlp = spacy.load("sv_core_news_sm")

    regexp4 = r'\b(?<![\\\.])\w+(?!\.\w+)\b'
    

    token_list = []
    # Remove unicode and webpages
    
    # Remove html tags, 'lxml' more robust than 'html.parser'
    # Requires pip install lxml
    result = BeautifulSoup(input, 'lxml').get_text()
    
    # Save words that are not preceded by a '\' and webpages
    result = re.findall(regexp4, result)
    

    # Remove excessive whitespaces
    result = " ".join(result)
    doc = nlp.pipe([result])
    for token in doc:
        token_list.append([t.lemma_.lower() for t in token if not t.is_stop 
                                                    and t.is_alpha])
    result = " ".join(token_list[0])
    print(result)
    return result

In [603]:
from os.path import exists

if(not exists('preprocessed_articles.csv')):  
    df_articles = pd.read_csv('sävsjö_articles.csv')
    df_articles = df_articles.dropna().reset_index(drop=True)
    # columns: ['ID', 'Title', 'Text']
    # iterate over 'Title' and 'Text' columns and preprocess the texts
    for col in df_articles.columns[1:]:
        df_articles[col] = df_articles[col].astype(str)
        df_articles[col] = df_articles[col].apply(lambda x: preprocess_input(x))
        #df_articles[col] = prep_input(df_articles[col])
        #df_articles[col] = preprocess_input(df_articles[col])
    
else:
    df_processed = pd.read_csv('preprocessed_articles.csv')

aktiespararna gunvorbudet
fast timme grop
mannen fastna grop timme
motorcykel
stulna braskamin värma
sävsjö ny skolchef
krösatåg försena
sävsjöpensionär uppra
linus testa
linus testa
linus test
test
linus
test
testing
dagisinbrott vecka
sävjöborna köra bil
liter diesel stängsl
tyskt teleföretag bluffakturera vrigstad
tuffa brud lyxförpackning
julskyltning val lucia


KeyboardInterrupt: 

In [ ]:
df_articles.head()

In [ ]:
def remove_labels(doc, labels:list):
        text_no_locations = ''
        for token in doc:
            if token.ent_type_ not in labels:
                text_no_locations += token.text
                if token.whitespace_:
                    text_no_locations += ' '
        return text_no_locations

def retrieve_tokens(doc):
    return [token.lemma_.lower() for token in doc if not token.is_stop 
                                                     and token.is_alpha]
    #return [token.lemma_.lower() for token in doc if token.pos_ not in pos_to_remove 
    #                                                                    and not token.is_stop 
    #                                                                    and token.is_alpha 
    #                                                                    and token.ent_type_ not in labels_to_remove]


In [ ]:
#from spacy.pipeline.ner import EntityRecognizer
from matplotlib.pyplot import text
from numpy import dtype
import spacy
from os.path import exists
np.warnings.filterwarnings('error', category=np.VisibleDeprecationWarning) 

if(not exists('preprocessed_articles.csv')):
    df_processed = df_articles.drop(columns=['Title']).copy()

    nlp = spacy.load("sv_core_news_sm")
    labels_to_remove = ['LOC', 'TME', 'MSR']
    pos_to_remove = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
    
    entity_list = []
    label_list = []
    unique_token_list = []
    token_list = []
    """
    for row in nlp.pipe(df_processed['Text']):
        label_free_text = remove_labels(row, labels_to_remove)
        row_tokens = [token.lemma_.lower() for token in row if token.pos_ not in pos_to_remove 
                                                                            and not token.is_stop 
                                                                            and token.is_alpha 
                                                                            and token.ent_type_ not in labels_to_remove]
        
        unique_ents = list({keyword.__repr__(): keyword for keyword in row.ents}.values())
        unique_ents = [ent for ent in unique_ents if not ent.label_ in labels_to_remove]
        entities = [str(x) for x in unique_ents]
        
        labels = [str(labels.label_) for labels in unique_ents]
        entity_list.append(entities)
        label_list.append(labels)
        tokens.append(row_tokens)
        text_list.append(label_free_text)
        #print(unique_ents)
        
    df_processed['Entities'] = pd.Series(entity_list)
    df_processed['Labels'] = pd.Series(label_list)
    df_processed['Tokens'] = pd.Series(tokens)
    df_processed['Text'] = pd.Series(text_list)
    """

    for index, row in df_articles.iterrows():
        
        doc_text = nlp(row['Text'])
        doc_title = nlp(row['Title'])
        
        doc_ents = doc_text.ents + doc_title.ents
        #text_tokens = retrieve_tokens(doc_text, pos_to_remove, labels_to_remove)
        #title_tokens = retrieve_tokens(doc_title, pos_to_remove, labels_to_remove)
        print(doc_text)
        title_tokens = retrieve_tokens(doc_title)
        text_tokens = retrieve_tokens(doc_text)
        print(text_tokens)
        doc_tokens = title_tokens + text_tokens
        unique_doc_tokens = set(doc_tokens)
        # Remove duplicate entities from the list
        unique_ents = list({keyword.__repr__(): keyword for keyword in doc_ents}.values())
        
        #entity_list.add(str(unique_ents))
        # Remove entities that have label 'TME' or 'LOC' because they are mostly redundant
        # And the text will be changed to not contain words with these labels
        unique_ents = [ent for ent in unique_ents if not ent.label_ in labels_to_remove]    
           
        entities = [str(x) for x in unique_ents]
        labels = [str(labels.label_) for labels in unique_ents]
        
        # Remove words that contain the specified labels
        label_free_text = remove_labels(doc_text, labels=labels_to_remove)

        entity_list.append(entities)
        label_list.append(labels)
        token_list.append(doc_tokens)
        unique_token_list.append(unique_doc_tokens)
        
        df_processed.at[index,['Text']] = label_free_text
        
        if index > 1:
            break
        
    df_processed['Entities'] = pd.Series(entity_list)
    df_processed['Labels'] = pd.Series(label_list)
    df_processed['Tokens'] = pd.Series(token_list)
    df_processed['Unique_Tokens'] = pd.Series(unique_token_list)
    #df_processed.to_csv('preprocessed_articles.csv')

df_processed.head(10)

# pipeline takes 6m 11s


In [ ]:
df_processed['Text'][5]

# remove empty rows and convert to lists.

### Not used so far

In [ ]:
""" Check for a value in a dataframe column"""
def check_value(df, column, value):
    return df[df[column].str.contains(value, na=False)]
check_value(df_processed, 'Entities', 'Smålandsvillan')

In [ ]:
"""
df_test = df_articles[["Labels","Entities"]]

for ind, row in df_test.iterrows():
    if row["Labels"] == "[]":
        df_test = df_test.drop(index=ind)

df_test[["Labels","Entities"]].apply(lambda x : str(x).split(','))
"""